In [1]:
import pandas as pd
import datetime
import json
import pandas_datareader as pdr
import numpy as np
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
from matplotlib.image import imread
from keras.preprocessing import image
from helper import data_to_tensor
from keras.applications import ResNet50
from keras import optimizers
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
cnn_base = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

C:\Users\FlatIron_User\.conda\envs\learn-env12_12_19\lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [3]:
cnn_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [4]:
model = models.Sequential()
model.add(cnn_base)
model.add(layers.Flatten())
model.add(layers.Dense(32))

model.add(layers.Dense(8, kernel_initializer='normal', activation = 'linear'))


In [5]:
model.layers[0].trainable = False
model.layers[1].trainable = False

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                3211296   
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264       
Total params: 26,799,272
Trainable params: 3,211,560
Non-trainable params: 23,587,712
_________________________________________________________________


In [7]:
image_list= [f'C:/Users/FlatIron_User/capstone_stock/ITCI_{i}.png' for i in range(60,1000)]
    

In [8]:
with open('stock_info.json', 'r') as f:
    data = json.load(f)

In [9]:
stocks = [ stock for stock in data.keys()]

#### Next Day Max/Min Value  (% Difference from Closing Price)

In [10]:
start = datetime.datetime.now()
period = [1, 2]
for stock in stocks:
    
    stock_dict_list = data[stock]
    num_days = len(stock_dict_list)
    for index, dict_ in enumerate(stock_dict_list):
        _open = dict_['Open']
        if index+period[1] >= num_days:
            _low = None
            _max = None
        else:
            _low = np.min([dict_['Low'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
            _max = np.max([dict_['High'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
        stock_dict_list[index]['Low_1'] = _low
        stock_dict_list[index]['Max_1'] = _max
    data[stock] = stock_dict_list

#### 3 Day Max/Min Value (% Difference from Closing Price)
The range is from the 2nd day to the 3rd after the input day

In [11]:
start = datetime.datetime.now()
period = [2, 4]
for stock in stocks:
    
    stock_dict_list = data[stock]
    num_days = len(stock_dict_list)
    for index, dict_ in enumerate(stock_dict_list):
        _open = dict_['Open']
        if index+period[1] >= num_days:
            _low = None
            _max = None
        else:
            _low = np.min([dict_['Low'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
            _max = np.max([dict_['High'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
        stock_dict_list[index]['Low_3'] = _low
        stock_dict_list[index]['Max_3'] = _max
    data[stock] = stock_dict_list
            

#### 10 Day Max/Min Value (% Difference from Closing Price)
The range is from the 4th day to the 10th after the input day

In [12]:
start = datetime.datetime.now()
period = [4, 11]
for stock in stocks:
    
    stock_dict_list = data[stock]
    num_days = len(stock_dict_list)
    for index, dict_ in enumerate(stock_dict_list):
        _open = dict_['Open']
        if index+period[1] >= num_days:
            _low = None
            _max = None
        else:
            _low = np.min([dict_['Low'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
            _max = np.max([dict_['High'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
        stock_dict_list[index]['Low_10'] = _low
        stock_dict_list[index]['Max_10'] = _max
    data[stock] = stock_dict_list
            

#### 30 Day Max/Min Value (% Difference from Closing Price)
The range is from the 11th day to the 30th after the input day

In [13]:
start = datetime.datetime.now()
period = [11, 31]
for stock in stocks:
    
    stock_dict_list = data[stock]
    num_days = len(stock_dict_list)
    for index, dict_ in enumerate(stock_dict_list):
        _open = dict_['Open']
        if index+period[1] >= num_days:
            _low = None
            _max = None
        else:
            _low = np.min([dict_['Low'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
            _max = np.max([dict_['High'] for dict_ in stock_dict_list[index+period[0]:index+period[1]]])/ _open -1
        stock_dict_list[index]['Low_30'] = _low
        stock_dict_list[index]['Max_30'] = _max
    data[stock] = stock_dict_list
            

#### Creating List of Dictionary to convert to a Dataframe 

In [14]:
data_in_list = []
for stock in stocks:
    stock_dict_list = data[stock]
    for index, dict_ in enumerate(stock_dict_list):
        dict_['Ticker'] = stock
        data_in_list.append(dict_)
df = pd.DataFrame(data_in_list)   
print(len(df))
df.head()

148088


,Adj Close,Close,Date,High,Low,Low_1,Low_10,Low_3,Low_30,Max_1,Max_10,Max_3,Max_30,Open,Ticker,Volume
0,15.000000,15.000000,2014-01-07 00:00:00,17.00,15.000000,0.000000,0.125000,0.068750,-0.012500,0.125000,0.250000,0.250000,0.250000,16.0,ITCI,59800.0
1,17.000000,17.000000,2014-01-08 00:00:00,18.00,16.000000,-0.022857,0.028571,0.000000,-0.097143,0.014286,0.142857,0.142857,0.142857,17.5,ITCI,51100.0
2,17.500000,17.500000,2014-01-09 00:00:00,17.75,17.100000,0.000000,0.028571,0.028571,-0.097143,0.142857,0.142857,0.100000,0.142857,17.5,ITCI,159900.0
3,18.000000,18.000000,2014-01-10 00:00:00,20.00,17.500000,0.040000,0.040571,0.028571,-0.097143,0.057143,0.142857,0.100000,0.142857,17.5,ITCI,157100.0
4,18.200001,18.200001,2014-01-13 00:00:00,18.50,18.200001,-0.021739,-0.002717,-0.021739,-0.141304,0.046196,0.086957,0.032609,0.086957,18.4,ITCI,5000.0


In [15]:
target = np.array(df.iloc[60:1000].loc[df.Ticker == 'ITCI'][['Low_1', 'Low_10', 'Low_3', 'Low_30', 'Max_1', 'Max_10', 'Max_3', 'Max_30']])

In [16]:
np.shape(target)

(940, 8)

In [17]:
np.shape(image_list)

(940,)

In [18]:
image_tensors = []
for image_location in image_list:
    img = image.load_img(image_location, target_size=(224, 224))
    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    #Follow the Original Model Preprocessing
    img_tensor /= 255.
    image_tensors.append(img_tensor)
   

In [19]:
np.shape(image_tensors)

(940, 1, 224, 224, 3)

In [20]:
image_tensors = np.array(image_tensors)


In [21]:
image_tensors = image_tensors.reshape((940,224,224,3))

In [22]:
model.input

<tf.Tensor 'resnet50_input:0' shape=(?, 224, 224, 3) dtype=float32>

In [23]:
model.output

<tf.Tensor 'dense_2/BiasAdd:0' shape=(?, 8) dtype=float32>

In [24]:
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mean_squared_error'])

In [ ]:
hist = model.fit(image_tensors, target, batch_size=10,
                         epochs=100, validation_split=0.1)


Train on 846 samples, validate on 94 samples
Epoch 1/100
400/846 [=============>................] - ETA: 3:30 - loss: 13.9819 - mean_squared_error: 13.9819